In [71]:
# ChatGPT
# a sequence of tokens, how they are sequenced, we give start of sequence the model continues
# Generatively Pre-trained Transformer
# Character level language model
# Tiny shakesphere - concatenated work of all shakesphere in text filter
# We model how charcters follow if given context
# Output is a shakesphere like text
# NanoGPT . train GPT with any text 124 million params - 38 hours training time in a single 8xA100 40GB Node
#  8xA100 40GB - One machine with 8 NVIDIA A100 GPUs, each with 40 GB of VRAM, connected via NVLink or PCIe for multi-GPU training
# using OpenWebText data 
# OpenwebText has 38GB of text data (40GB using SI units) from 8,013,769 documents
# train.py is 300 line training loop
# model.py a 300 line GPT model definition
# A multilayer perceptron is a fully connected feedforward net with at least three layers (input, hidden and output)


In [72]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [73]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [74]:
print(f"length of dataset in characters:  {len(text)}, type of dataset: {type(text)}")

length of dataset in characters:  1115394, type of dataset: <class 'str'>


In [75]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
set_of_text = set(text) # {'e', 'X', 'y', 'N', 'J', '3', 'o', 'w', 'b', 'U', '
list_of_text = list(set_of_text) # ['e', 'X', 'y', 'N', 'J', '3', 'o', 'w', 'b', 'U', 
print(chars) # ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size) # entire set of characters - all possible characters the model can see

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [76]:
# tokenize - convert raw text to sequence of numbers based on vocabulary. this is char level tokenization. all chars
# openi - bite pair encoding . 50000 tokens- hii there - integers between 0 to 50000. sub word encoding. short seq int with large vocab or large seq integer with small vocab
# create a mapping from characters to integers
# enumerate(iterable, start=0)
# Using the enumerate() function is useful from the standpoint of its memory and computing efficiency compared to using for-loops since it returns the index and the corresponding item at one go.
# this function assigns a count incrementing by 1 to each item of an iterable and helps us track iterations while looping through that object
stoi = { ch:i for i,ch in enumerate(chars) } #  enumerate(chars). Lookup table Each char gets a number {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4
itos = { i:ch for i,ch in enumerate(chars) } #. Reverse lookup table of itos {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'" .. Each number gets a character
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers. For each char, outputs the corresponding number
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string. For each number outputs the correponding character and then combined as a string
print(stoi)
print(itos)
print(encode("hii there")) # list of integers that represent string
print(decode(encode("hii there"))) # reverse mapping, decode integers



{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [77]:
import torch
data = torch.tensor(encode(text), dtype=torch.long) # text is that massive string, now it outputs a tensor a 1-1 encoding of the characters in text to numbers
print(data.shape, data.dtype) # shape is 1 x 1115394 # torch.Size([1115394]) torch.int64
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [78]:
# train validation split
n = int(0.9*len(data))
train_data = data[:n]
val_data=data[n:]
# we dont feed all at once, we sample random chunks and train on chunks at once
# max chunk is blocksize

In [79]:
# we dont feed all at once, we sample random chunks and train on chunks at once
# max chunk is blocksize 
block_size = 8
train_data[:block_size+1] # tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]). context of 18, 47 56 comes next, 18,47,56 context 57 comes next

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [80]:
x = train_data[:block_size] # 8 characters
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1] # for 0 to t characters, target is the corresponding t
    target = y[t]
    print(f"when input is {context} the target: {target}")

# we could predict from 1 to block size, beyond this we have to truncate
# mini batches multiple chunks stacked on top of each other

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [81]:
torch.manual_seed(1337) # seed is set so it is reproducable
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generate batch size number of random offsets# gets 4 random indices between 0 to len(data) - block_size as we need mini batches too
    x = torch.stack([data[i:i+block_size] for i in ix]) # gets 8 chars from indices. stack as rows  4x8 tensor
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # gets 8 chars  from index+1
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension # for each mini batch 
    for t in range(block_size): # time dimension
        context = xb[b, :t+1] # does the context step 0 - 0: 0,1- 1:0,1,2-2
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
# #
# target = yb[b, t] → the token the model should predict at this position

# Because yb is shifted by 1 from xb, the target is always the next token after the current context.

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [82]:
# Bigram model

import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # a lookup table 65 number of embeddings, and 65 is embedding dim
        print("before", self.token_embedding_table)
        '''we have 65 possible tokens (chars in dataset)
        Each token is mapped to a vector of size 65.
        E = [
            [e11, e12, ..., e165],   # embedding vector for token 0
            [e21, e22, ..., e265],   # embedding vector for token 1
            ...
            [e651, ..., e6565]       # embedding vector for token 64
            ]

        '''
    def forward(self, idx, targets=None):
        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) 
        # the vocab size is 65- all characters of shakesphere. we create a random embedding of 65 x 65 . 
        # where 65 is vocab size and 65 is the embedding space. They are learnable weights
        # During training, they get adjusted so that tokens that appear in similar contexts have similar embeddings.
        # The order of tokens will matter downstream when predicting the next token, because the model sees xb[b, :t] as “context”.
        # But the embedding of each token by itself does not encode position — it’s just a vector representing that token.
        # print(logits)
        '''
        We have the input as 4 x 8 of the forward encode - and it gives the embedding for each
        ''' 
        if targets is None:
            loss = None
        else:
            # picks out the row from table B,T,C 4 x 8 x 65
            B, T, C = logits.shape
            logits = logits.view(B*T, C ) # we stack the tokens neatly one below another without messing up the order. so we have 32 rows each of 65 numbers as per embedding
            targets = targets.view (B*T) # 4 x 8 gives (32,)
            loss = F.cross_entropy(logits, targets) # torch wants B x T X C. 
            # For each row of logits, Treat it as the unnormalized scores for all 65 tokens. Apply softmax internally → get probabilities over 65 classes
            # for embeddings of other size 128 dimension etc, we have a linear neural network that projects it back to logits
            # self.fc = nn.Linear(embedding_dim, vocab_size)  # 124 → 65
            # logits = self.fc(embedding_table[idx])          # shape: (B, T, 65)
            # omg that Problem: You cannot feed 124-dimensional embeddings directly to cross-entropy, because cross_entropy expects logits over vocab size (65).
            # Solution: add a linear projection: totally solved all my confusion. karpathy basically left this critical step, or he had another video which i 
            # didnt watch. so basically the embedding is like a input for neural network with its own layers . can be multiple and that outputs the logits and we do 
            # softmax to get max probability of token and compare that to target. 
            # solved. all clear now only tell me in this case the input is independently used to generate output token by token 
            # 24, 43, 58,  5, 57,  1, 46, 43 - are we brute forcing it. like for ex  we use embedding of 58 only to generate the output and not 24,43,58
            # The model is very simple: it only looks at the current token to predict the next token
            # Input 24 → embedding → predicts next token 43
            # Input 43 → embedding → predicts next token 58
            # Input 58 → embedding → predicts next token 5
            # … and so on
            # ✅ So yes, each step is independent in terms of what embedding is used: only the current token’s embedding is fed into the model to predict the next token.
                        
        return logits, loss 

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # goes to forward function
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) # pick out only the last element of a B x T  x C logit a  (B, C) = (4, 65) tensor if input is 4 x 8 x 65
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C) # logits to softmax. dim=-1 → apply softmax along the last dimension (here, C, vocab dimension)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) # get only one
            # torch.multinomial samples from a categorical distribution defined by probs
            # we want the model’s prediction to guide the choice: tokens with higher probabilities are more likely to be picked.
            # probs = [0.7, 0.1, 0.1, 0.1]  # vocab size = 4
            # torch.multinomial(probs, 1)   # 70% chance to pick token 0, 10% chance for 1,2,3
            # If you always picked argmax, it would be deterministic. Multinomial allows stochastic sampling, which is important for generating diverse tex
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) concatenate along seq dimension dim=1 → new idx is (B, T+1) = (4, 9)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) # a forward pass with 4 x 8 matrix
# tensor([[24, 43, 58,  5, 57,  1, 46, 43],
#         [44, 53, 56,  1, 58, 46, 39, 58],
#         [52, 58,  1, 58, 46, 39, 58,  1],
#         [25, 17, 27, 10,  0, 21,  1, 54]])
# targets:
# torch.Size([4, 8])
# tensor([[43, 58,  5, 57,  1, 46, 43, 39],
#         [53, 56,  1, 58, 46, 39, 58,  1],
#         [58,  1, 58, 46, 39, 58,  1, 46],
#         [17, 27, 10,  0, 21,  1, 54, 39]])
print(loss) # we get scores for torch.Size([4, 8, 65])
idx = torch.zeros((1,1), dtype=torch.long) # 0 is a newline character and we start 
idx = torch.tensor([[5]]) # 5 as the starting
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))
# this is without history
# bigram model. this looks only at last character to predict next. we expand to use history to predict next token


before Embedding(65, 65)
tensor(5.0364, grad_fn=<NllLossBackward0>)
'XAQHukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs
lZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH


In [83]:

## training this model
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # for smaller network higher LR 


In [84]:
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# print(device)

In [ ]:

import time

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print("Using device:", device)

# Move model to GPU
m = m.to(device)

batch_size = 128  # bigger batch helps GPU utilization
steps = 10000

# Optional: wrap get_batch to move to GPU automatically
def get_batch_device(split):
    xb, yb = get_batch(split)
    return xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)

start_time = time.time()
for step in range(steps):
    xb, yb = get_batch_device('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        # print occasionally to avoid slowing down GPU
        print(f"Step {step}, Loss: {loss.item():.4f}")

end_time = time.time()
print(f"Training finished in {end_time - start_time:.2f} seconds")
# 2 min 34 secs for cuda 4.6973 to 2.4142. 2 min 20.3 secs in CPU 4.6973 to 2.4142


Using device: cpu
Step 0, Loss: 4.6973


Step 100, Loss: 4.4980
Step 200, Loss: 4.3540
Step 300, Loss: 4.3019
Step 400, Loss: 4.1383
Step 500, Loss: 4.0842
Step 600, Loss: 3.9211
Step 700, Loss: 3.8928
Step 800, Loss: 3.7173
Step 900, Loss: 3.6722
Step 1000, Loss: 3.5917
Step 1100, Loss: 3.4900
Step 1200, Loss: 3.4401
Step 1300, Loss: 3.3894
Step 1400, Loss: 3.3141
Step 1500, Loss: 3.2329
Step 1600, Loss: 3.1788
Step 1700, Loss: 3.1531
Step 1800, Loss: 3.0930
Step 1900, Loss: 3.0790
Step 2000, Loss: 2.9576
Step 2100, Loss: 2.9700
Step 2200, Loss: 2.8952
Step 2300, Loss: 2.8947
Step 2400, Loss: 2.8516
Step 2500, Loss: 2.8409
Step 2600, Loss: 2.8163
Step 2700, Loss: 2.6917
Step 2800, Loss: 2.7242
Step 2900, Loss: 2.6964
Step 3000, Loss: 2.6581
Step 3100, Loss: 2.6654
Step 3200, Loss: 2.6539
Step 3300, Loss: 2.6368
Step 3400, Loss: 2.6182
Step 3500, Loss: 2.5721
Step 3600, Loss: 2.5843
Step 3700, Loss: 2.5734
Step 3800, Loss: 2.5613
Step 3900, Loss: 2.5380
Step 4000, Loss: 2.6231
Step 4100, Loss: 2.5287
Step 4200, Loss: 2.5319
S

In [ ]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))
#initially the output is 
# 'XAQHukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs
# lZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH

# after 10000 steps it is 
# I yome k ee mprop ves.
# med belenk s w me ave hirmin, baveard.
# bellfushealmiere ENus
# BAy he,
# NARWhan 


I yome k ee mprop ves.
med belenk s w me ave hirmin, baveard.
bellfushealmiere ENus
BAy he,
NARWhan 


In [86]:
# batch_size = 32
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# m = m.to(device)
# for steps in range(10000):
#     # sample a batch
#     xb, yb = get_batch('train')
#     xb = xb.to(device)
#     yb = yb.to(device)
#     # training loop
#     logits, loss = m(xb, yb)
#     optimizer.zero_grad(set_to_none=True) # zeroing up the gradients in previous step
#     loss.backward()
#     optimizer.step() # update parameters
#     print(loss.item())
# # loss goes from for 3.32 to 3.27 in 100 iterations
# # 10,000 steps need 1 min 7.5 seconds in CPU. with cuda it is 1 min 11.2 secs for 10,000 from start. loss went from 4.67 to 2.54

In [89]:
# The tokens are not taking to each other. now they talk to each other and figure out what is in context and predict better
# estimate loss in multiple batches - trainand validation loss. torch.no_grad() in estimate loss
# token in 5th location should talk only to 1,2,3,4 and not 6,7,8
# 5th token - 1,2,3,4 averaged out info like a feature. every t tokens calculate avg of vectors
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [90]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B): # for each  batch
    for t in range(T): # for each block from first to cumulative of last 
        xprev = x[b,:t+1] # (t,C) always a t+1 x c where t ranges from 0 to some number 
        xbow[b,t] = torch.mean(xprev, 0) # torch.mean(xprev, 0) computes the mean along the first dimension (rows). It sums up each column of xprev (element-wise across the 4 rows).
        #Then divides by the number of rows (t+1 = 4) → gives average per column.

In [92]:
print(x[0])
print(xbow[0])
# tensor([[ 0.1808, -0.0700],
#         [-0.3596, -0.9152],
#         [ 0.6258,  0.0255],
#         [ 0.9545,  0.0643],
#         [ 0.3612,  1.1679],
#         [-1.3499, -0.5102],
#         [ 0.2360, -0.2398],
#         [-0.9211,  1.5433]])
# tensor([[ 0.1808, -0.0700],
#         [-0.0894, -0.4926],
#         [ 0.1490, -0.3199],
#         [ 0.3504, -0.2238],
#         [ 0.3525,  0.0545],
#         [ 0.0688, -0.0396],
#         [ 0.0927, -0.0682],
#         [-0.0341,  0.1332]])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
# torch.ones give 3 x 3 of 1 s
# troch.tril gives a lower quadrant 1s and upper zeros
a = a / torch.sum(a, 1, keepdim=True)
# this gives the avg of each row in place of 1s, first row has one 1 so it is 1, second row has two 1s so 0.5 , 0.5 each row adds up to 1. so in this way our matrix 
# multiplication gives avg till that number , basically what we did using for loop above
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T)) # using the above concept we get T x T - each row
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C). Torch auto adds B times T x T to wei 
torch.allclose(xbow, xbow2)

In [ ]:
# version 3: use Softmax
# weighted aggregation of lower triangle. the -inf mask is to ensure future tokens are not communicating with past when we do weighted average
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)